In [ ]:
!pip install -q rasterio

In [ ]:
# Bibliotecas
import rasterio as rio
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go

In [ ]:
# Carregar as imagens classificadas
with rio.open('/content/mapbiomas-brazil-collection-90-altamirapa1500602-2013.tif') as src_2013:
    img_2013 = src_2013.read(1)

with rio.open('/content/mapbiomas-brazil-collection-90-altamirapa1500602-2023.tif') as src_2023:
    img_2023 = src_2023.read(1)

# Calcular a matriz de transição
classes_2013 = np.unique(img_2013)
classes_2023 = np.unique(img_2023)

matriz_transicao = {}

for classe_2013 in classes_2013:
    for classe_2023 in classes_2023:
        # Contar o número de pixels que mudaram de classe_2013 para classe_2023
        count = np.sum((img_2013 == classe_2013) & (img_2023 == classe_2023))
        if count > 0:
            matriz_transicao[(classe_2013, classe_2023)] = count

In [ ]:
# Dicionário com descrições das classes
class_descriptions = {
    1: "Forest",
    3: "Forest Formation",
    4: "Savanna Formation",
    5: "Mangrove",
    6: "Floodable Forest",
    49: "Wooded Sandbank Vegetation",
    10: "Herbaceous and Shrubby Vegetation",
    11: "Wetland",
    12: "Grassland",
    32: "Hypersaline Tidal Flat",
    29: "Rocky Outcrop",
    50: "Herbaceous Sandbank Vegetation",
    14: "Farming",
    15: "Pasture",
    18: "Agriculture",
    19: "Temporary Crop",
    39: "Soybean",
    20: "Sugar cane",
    40: "Rice",
    62: "Cotton (beta)",
    41: "Other Temporary Crops",
    36: "Perennial Crop",
    46: "Coffee",
    47: "Citrus",
    35: "Palm Oil",
    48: "Other Perennial Crops",
    9: "Forest Plantation",
    21: "Mosaic of Uses",
    22: "Non vegetated area",
    23: "Beach",
    24: "Urban Area",
    30: "Mining",
    25: "Other non Vegetated Areas",
    26: "Water",
    33: "River",
    31: "Aquaculture",
    27: "Not Observed"
}

In [ ]:
# Criar listas para os dados do Sankey, excluindo a classe zero
source = []
target = []
value = []

# Dicionário para mapear as classes para índices separados para 2013 e 2023
class_indices_2013 = {}
class_indices_2023 = {}
index_counter = 0

# Definir índices para classes de 2013
for classe in classes_2013:
    if classe != 0:
        class_indices_2013[classe] = index_counter
        index_counter += 1

# Definir índices para classes de 2023
for classe in classes_2023:
    if classe != 0:
        class_indices_2023[classe] = index_counter
        index_counter += 1

# Preencher listas source, target e value para o Sankey
for (classe_2013, classe_2023), count in matriz_transicao.items():
    if classe_2013 != 0 and classe_2023 != 0:  # Ignorar a classe zero
        source.append(class_indices_2013[classe_2013])
        target.append(class_indices_2023[classe_2023])
        value.append(count)

# Criar labels para as classes com suas descrições
labels_2013 = [f"{class_descriptions.get(classe, f'Class {classe}')}" for classe in class_indices_2013.keys()]
labels_2023 = [f"{class_descriptions.get(classe, f'Class {classe}')}" for classe in class_indices_2023.keys()]
labels = labels_2013 + labels_2023

# Criar o diagrama de Sankey
fig = go.Figure(go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=labels
    ),
    link=dict(
        source=source,
        target=target,
        value=value
    )
))

fig.update_layout(title_text="Diagrama de Sankey das Mudanças de Classe (2013-2023)", font_size=10)
fig.show()
